In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        
        # Use OUTPUT_SIZE = (INPUT_SIZE-FILTER+2PADDING)/STRIDE + 1)
        # Here INPUT_SIZE AND OUTPUT_SIZE are based on dimesions of image, rather than Conv2d first and second arguments
        # 4*4 occurs because starting dimension was assumed to be 32*32
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 7)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 64*4*4)
        x = F.dropout(x, 0.25)
        
        x = F.softmax(x, dim=1)
        
        return x

model = Network()
model

Network(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

In [43]:
# Batch size of 5, with 3x32
input_tensor = torch.randn((5, 3, 32, 32))
input_tensor

tensor([[[[ 6.6652e-01, -1.6534e+00, -1.7198e-01,  ...,  2.1165e-01,
            5.8160e-01, -2.0809e+00],
          [ 2.5532e-01, -4.7830e-01,  4.8699e-01,  ...,  5.5077e-01,
            1.5959e+00, -1.9193e-01],
          [ 4.5434e-03,  5.7492e-01, -9.3109e-01,  ..., -3.4775e-01,
            5.4940e-01,  1.5178e-01],
          ...,
          [ 7.6290e-01, -9.1081e-01, -4.5836e-01,  ...,  1.1445e+00,
           -1.3330e+00,  6.7047e-01],
          [ 7.3255e-01, -3.0696e-01,  7.5945e-01,  ..., -1.3193e+00,
            6.1936e-02,  6.8189e-01],
          [-5.2722e-01, -2.3824e+00,  1.7133e-01,  ...,  9.8521e-01,
           -7.2145e-01, -5.7927e-01]],

         [[ 1.3058e+00,  4.0273e-01,  1.4033e+00,  ..., -7.1425e-02,
           -8.6371e-01, -9.2461e-01],
          [-6.1982e-01,  7.4960e-01, -1.0590e+00,  ..., -1.9074e-01,
           -3.5547e-01, -2.8177e-01],
          [ 4.3474e-02, -2.2330e-01, -2.1807e-01,  ...,  1.3253e+00,
            8.1071e-01, -1.2492e-01],
          ...,
     

In [44]:
model(input_tensor)

tensor([[0.0011, 0.0011, 0.0012,  ..., 0.0008, 0.0008, 0.0009],
        [0.0008, 0.0008, 0.0008,  ..., 0.0008, 0.0008, 0.0008],
        [0.0012, 0.0014, 0.0008,  ..., 0.0008, 0.0008, 0.0008],
        [0.0011, 0.0012, 0.0008,  ..., 0.0008, 0.0008, 0.0008],
        [0.0010, 0.0012, 0.0013,  ..., 0.0008, 0.0008, 0.0008]],
       grad_fn=<SoftmaxBackward>)